In [1]:
# initialize findspark for local
import findspark
findspark.init('/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7')

In [2]:
import configparser
from datetime import datetime
import os
import sys
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType, LongType

In [88]:
import spark.implicits._

ModuleNotFoundError: No module named 'spark'

In [3]:
# read config file
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.1 pyspark-shell'

In [4]:
# set paths to get to pyspark and java
pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, "/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip"))

# Initialize PySpark
exec(open(os.path.join(spark_home, "/Users/johnrick/Downloads/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py")).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
SparkSession available as 'spark'.


In [ ]:
# spark-submit --packages org.apache.hadoop:hadoop-aws:2.7.0

In [ ]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:2.7.0'

In [ ]:
#  --packages com.amazonaws:aws-java-sdk-pom:1.11.760,org.apache.hadoop:hadoop-aws:2.7.0 --conf spark.hadoop.fs.s3a.endpoint=s3.us-west-2.amazonaws.com

In [ ]:
# config = configparser.ConfigParser()
# config.read('dl.cfg')

# os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [ ]:
# # store s3 bucket location
# input_data = "s3://udacity-dend/song_data"

# song_data = input_data + "/*/*/*/*.json"

# Create S3 Bucket

## create boto3 session

In [5]:
import logging
import boto3
from botocore.exceptions import ClientError

In [6]:
target_region = "us-west-2"

In [7]:
s3 = boto3.resource('s3',
                    region_name=target_region,
                   aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'],
                   aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'])

In [8]:
def create_bucket(bucket_name, region=None, acl="private"):
    """
    Create an S3 bucket in a specified region
    
    Args:
        bucket_name: Bucket to create
        region: String region to create bucket in, e.g., 'us-west-2'
        acl: Canned access control list to apply to the bucket. 'public-read'
            makes sure everything posted is publicly readable
    """
    
    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration=location,
                ACL=acl
            )
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [9]:
#create bucket from jupyter notebook so credentials can be accepted
create_bucket('ud1-s3datalake', target_region, acl='public-read')

True

# Get Info From s3 Bucket

In [118]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
spark.conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

In [16]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://ud1-s3datalake/"

In [109]:
song_data = input_data + "song_data/A/A/*/*.json"

In [110]:
song_data

's3a://udacity-dend/song_data/A/A/*/*.json'

In [111]:
song_schema = StructType([
        StructField("artist_id", StringType(), False),
        StructField("artist_latitude", StringType(), True),
        StructField("artist_longitude", StringType(), True),
        StructField("artist_location", StringType(), True),
        StructField("artist_name", StringType(), False),
        StructField("song_id", StringType(), False),
        StructField("title", StringType(), False),
        StructField("duration", DoubleType(), False),
        StructField("year", IntegerType(), False)
])

In [112]:
df = spark.read.json(song_data, schema=song_schema)

## songs_table insert

In [67]:
from pyspark.sql.functions import monotonically_increasing_id

In [113]:
songs_table= df.select("title",
         "artist_id",
         "year",
         "duration").dropDuplicates().withColumn("song_id", monotonically_increasing_id()).limit(25)

In [116]:
# extract columns to create songs table
songs_table = df.select("title", 
                       "artist_id",
                       "year",
                       "duration").dropDuplicates().withColumn("song_id", monotonically_increasing_id())

In [115]:
songs_table.limit(10).toPandas()

,title,artist_id,year,duration,song_id
0,Without You,ARQVORN11F50C4EFEC,0,165.38077,8589934592
1,Escape the U.K.,AROYS8V1187FB53D56,0,447.11138,8589934593
2,Master of Puppets,ARSRAKS11F4C83FDB5,2007,311.82322,17179869184
3,Love's Been Good To Me,ARXSPYZ1187B98972A,0,131.05587,17179869185
4,Never Be Nothing Like Me,ARPVZTA1187B9A63A8,0,262.24281,17179869186
5,Niev nietch nievitch,ARVEJ9M1187FB4DC44,1999,254.06649,17179869187
6,Afraid Of Everything And So On,AROO15N1187B99B64C,0,261.48526,17179869188
7,Rubberneck Lions (LP Version),ARX16TQ1187B9899C9,2001,315.53261,25769803776
8,Ain't No Secret,ARBTDXS11E2835CEC6,2009,209.34485,25769803777
9,A Favor,ARAZP5N1187B9B4D5D,2004,359.33995,25769803778


In [119]:
songs_table.write.parquet(output_data+"songs4.parquet", mode="overwrite", partitionBy=["year","artist_id"])

In [85]:
songs_table.limit(15).toPandas()

,title,artist_id,year,duration,song_id
0,I'll Slap Your Face (Entertainment USA Theme),ARSVTNL1187B992A91,2001,129.85424,68719476736
1,Game & Watch,AREWD471187FB49873,2004,580.54485,223338299392
2,Double Wide,ARKYKXP11F50C47A6A,0,160.20853,300647710720
3,Scream,ARJNIUY12298900C91,2009,213.94240,326417514496
4,I'll Be Waiting,ARCLYBR1187FB53913,1989,304.56118,386547056640
5,Drop of Rain,AR10USD1187B99F3F1,0,189.57016,429496729600
6,Drillbit,ARZKCQM1257509D107,0,374.62159,455266533376
7,A Poor Recipe For Civic Cohesion,AR73AIO1187B9AD57B,2005,118.07302,463856467968
8,It's About Time,ARC1IHZ1187FB4E920,0,246.98730,515396075520
9,Indian Angel,ARY589G1187B9A9F4E,2004,171.57179,523986010112


In [ ]:
#version 1
#songs_table.write.partitionBy("year", "artist_id").format("parquet").save("s3a://udacity-dend/songs")

In [70]:
songs_table.write.partitionBy("year", "artist_id").format("parquet").mode("overwrite").save(output_data + "songs")

## Artists Table

In [24]:
# extract column to get artists_table
artists_table = df.select("artist_id",
                         col("artist_name").alias("name"),
                         col("artist_location").alias("location"),
                         col("artist_latitude").alias("latitude"),
                         col("artist_longitude").alias("longitude")).dropDuplicates().limit(10)

In [26]:
# write artists table to parquet files
artists_table.write.format("parquet").mode("overwrite").save(output_data+"artists")


# Process Log Data

In [27]:
log_schema = StructType([
        StructField("artist", StringType(), True),
        StructField("auth", StringType(), False),
        StructField("firstName", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("itemInSession", IntegerType(), False),
        StructField("lastName", StringType(), True),
        StructField("length", DoubleType(), True),
        StructField("level", StringType(), False),
        StructField("location", StringType(), True),
        StructField("method", StringType(), False),
        StructField("page", StringType(), False),
        StructField("registration", DoubleType(), True),
        StructField("sessionId", IntegerType(), False),
        StructField("song", StringType(), True),
        StructField("status", IntegerType(), False),
        StructField("ts", DoubleType(), False),
        StructField("userAgent", StringType(), True),
        StructField("userId", StringType(), True)
    ])

In [28]:
output_data+"log_data/*/*/*.json"

's3a://ud1-s3datalake/log_data/*/*/*.json'

In [31]:
# get filepath to log data file
log_data = input_data +"log_data/*/*/*.json"

In [32]:
# read log data file
log_df = spark.read.json(log_data, schema = log_schema)

In [33]:
# filter by actions for song plays
log_df = log_df.where(log_df.page == "NextSong")

## users_table insert

In [37]:
users_table = log_df.select(col("userId").alias("user_id"),
                       col("firstName").alias("first_name"),
                       col("lastName").alias("last_name"),
                       "gender",
                       "level").dropDuplicates().limit(10)

In [36]:
output_data+"users"

's3a://ud1-s3datalake/users'

In [ ]:
"s3a://ud1-s3datalake/users"

In [95]:
users_table.write.format("parquet").mode("overwrite").save(output_data+"users.parquet")

## Time Stamp

In [39]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
log_df = log_df.withColumn("timestamp", get_timestamp(log_df.ts))

In [40]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))
log_df = log_df.withColumn("datetime", get_datetime(log_df.ts))

In [ ]:
# # extract columns to create time table
# time_table = df.select(
#     df.timestamp.alias('start_time'),
#     hour(df.datetime).alias('hour'),
#     dayofmonth(df.datetime).alias('day'),
#     weekofyear(df.datetime).alias('week'),
#     month(df.datetime).alias('month'),
#     year(df.datetime).alias('year'),
#     date_format(df.datetime, 'u').alias('weekday')).dropDuplicates()

In [41]:
# extract columns to create time table
time_table = log_df.select(
    log_df.timestamp.alias('start_time'),
    hour(log_df.datetime).alias('hour'),
    dayofmonth(log_df.datetime).alias('day'),
    weekofyear(log_df.datetime).alias('week'),
    month(log_df.datetime).alias('month'),
    year(log_df.datetime).alias('year'),
    date_format(log_df.datetime, 'u').alias('weekday')).dropDuplicates().limit(10)

In [42]:
output_data + "time"

's3a://ud1-s3datalake/time'

In [43]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").format("parquet").save(output_data + "time")

# songplays_table

## Load data from our s3

In [128]:
from time import time

In [136]:
def load_table(input_data, sub_data):
    print(f"Start loading dataframe from {sub_data}:")
    t0 = time()
    df = spark.read.parquet(input_data+sub_data)
    load_time = time()-t0
    print("=== DONE IN: {0:.2f} sec\n".format(load_time))
    return df
          

In [139]:
song_df = load_table(output_data, "songs4/*/*/*")

Start loading dataframe from songs4/*/*/*:
=== DONE IN: 40.23 sec



In [140]:
song_df.limit(5).toPandas()

,title,duration,song_id
0,Drown In My Own Tears (24-Bit Digitally Remast...,192.52200,1563368095744
1,I'll Slap Your Face (Entertainment USA Theme),129.85424,68719476736
2,King Of Scurf (2007 Digital Remaster),301.40036,670014898176
3,The Last Beat Of My Heart (b-side),337.81506,1580547964928
4,A Poor Recipe For Civic Cohesion,118.07302,463856467968


In [ ]:
df.limit(5).toPandas()

In [50]:
output_data+"songs2/*/*/*/*"

's3a://ud1-s3datalake/songs/*/*/*/*'

In [91]:
songs_s3 = output_data+"songs4/*/*/*"

In [ ]:
song_df = spark.read.format("parquet").load(output_data+"songs4/*/*/*")

In [ ]:
song_df.limit(5).toPandas()

In [102]:
# read in song data to use for songplays table
song_df = spark.read.parquet(output_data+"songs4/*/*/*.parquet")

In [103]:
song_df.limit(5).toPandas()

,title,duration,song_id
0,Drown In My Own Tears (24-Bit Digitally Remast...,192.52200,1563368095744
1,I'll Slap Your Face (Entertainment USA Theme),129.85424,68719476736
2,King Of Scurf (2007 Digital Remaster),301.40036,670014898176
3,The Last Beat Of My Heart (b-side),337.81506,1580547964928
4,A Poor Recipe For Civic Cohesion,118.07302,463856467968


In [61]:
artist_df = spark.read.parquet(output_data+"artists/*")

In [142]:
artist_df.limit(5).toPandas()

,artist_id,name,location,latitude,longitude
0,ARXR32B1187FB57099,Gob,,None,None
1,AR1KTV21187B9ACD72,Cristina,California - LA,34.05349,-118.24532
2,ARZKCQM1257509D107,Dataphiles,,None,None
3,ARY589G1187B9A9F4E,Talkdemonic,"Portland, OR",45.51179,-122.67563
4,ARSVTNL1187B992A91,Jonathan King,"London, England",51.50632,-0.12714


In [144]:
songs_logs = log_df.join(song_df, (log_df.song == song_df.title))

In [145]:
songs_logs.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,song,status,ts,userAgent,userId,timestamp,datetime,title,duration,song_id
0,Michael Jackson,Logged In,Kate,F,32,Harrell,278.282,paid,"Lansing-East Lansing, MI",PUT,...,Scream,200,1.542299e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-15 11:19:05,2018-11-15,Scream,213.9424,326417514496


In [146]:
artists_songs_logs = songs_logs.join(artist_df, (songs_logs.artist == artist_df.name))

In [147]:
artists_songs_logs.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,timestamp,datetime,title,duration,song_id,artist_id,name,location,latitude,longitude


In [ ]:
artists_songs_logs.columns

In [ ]:
songplays = artists_songs_logs.join(
        time_table,
        artists_songs_logs.timestamp == time_table.start_time, 'left')

In [ ]:
songplays.dropDuplicates().limit(2).toPandas()

In [ ]:
# extract columns to create songs table
songs_table = df.select("song_id",
                       "title", 
                       "artist_id",
                       "year",
                       "duration").dropDuplicates()

In [ ]:
#

# Using the data we created

In [ ]:
songs_table

In [ ]:
artists_table

In [ ]:
users_table

In [ ]:
time_table

### Test If we are loading correctly

In [ ]:
### Function for quality check
def validate_table_insertion(table, id):
    """
    Read the parquet file into dataframe, convert spark df to sql table
    Check the table using sql queries
    """
    ### load table
    table_name = os.path.join(input_path, table)
    parquetFile = spark.read.parquet(table_name)
    ### print count
    print('\033[1m' + f"Validate insertion from table {table}:" + '\033[0m')
    print("Dimension of the table:", parquetFile.count())
    
    ### convert df to sql table
    parquetFile.createOrReplaceTempView(table)
    
    ### count unique
    unique = spark.sql(f"SELECT COUNT(DISTINCT {id}) count FROM {table};")
    print("Number of unique id: ", unique.select('count').collect()[0]['count'])
    ### check null id
    
    null_id = spark.sql(f"SELECT COUNT(*) count FROM {table} WHERE {id} = NULL;")    
    print("Number of null id: ", null_id.select('count').collect()[0]['count'])
    
    ### print some examples
    print("Some examples from the table:")
    examples = spark.sql(f"SELECT * FROM {table} LIMIT 5;")
    examples.show()

In [ ]:
input_path

In [ ]:
input_path = 'output' # location of all the tables
tables = ['songs', 'artists', 'users', 'time', 'songplays']
ids = ["song_id", "artist_id", "user_id", "start_time", "start_time"]

### Perform all the check:
for i in range(0, len(tables)):
    validate_table_insertion(tables[i], ids[i])